In [1]:
import sys
sys.path.append('/home/npopkov/dll24')

import h5py
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
torch.set_float32_matmul_precision('medium')

class LatentDataset(Dataset):
    def __init__(self, h5_file):
        self.data = self.createData(h5_file)

    def createData(self, h5_file):
        data = []
        for key in h5_file.keys():
            data.append(np.array(h5_file[key]))
        
        return torch.tensor(np.array(data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

hdf = h5py.File('encoded_samples.hdf5', 'r')
dataset = LatentDataset(hdf)
hdf.close()

In [2]:
dataset.data[0].shape

torch.Size([256, 103])

In [3]:
from models.ddpm.DDPMBase import DDPMBase
from models.ddpm.CAUnet import CAUnet


n_steps, betaMin, betaMax = 200, 10**-6, 0.002

network = CAUnet(n_steps=n_steps,
                 c_in=256,
                 c_out=256,).to('cuda')

model = DDPMBase(network=network,
                betaMin=betaMin,
                betaMax=betaMax,
                n_steps=n_steps).to('cuda')

/home/npopkov/dll24/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from audio_diffusion_pytorch import DiffusionModel, UNetV0, VDiffusion, VSampler

# model = DiffusionModel(
#     net_t=UNetV0, # The model type used for diffusion (U-Net V0 in this case)
#     in_channels=256, # U-Net: number of input/output (audio) channels
#     channels=[8, 32, 64, 128, 256, 512, 512, 1024, 1024], # U-Net: channels at each layer
#     factors=[1, 4, 4, 4, 2, 2, 2, 2, 2], # U-Net: downsampling and upsampling factors at each layer
#     items=[1, 2, 2, 2, 2, 2, 2, 4, 4], # U-Net: number of repeating items at each layer
#     attentions=[1,1,1, 1, 1, 1, 1, 1, 1], # U-Net: attention enabled/disabled at each layer
#     attention_heads=16, # U-Net: number of attention heads per attention item
#     attention_features=64, # U-Net: number of attention features per attention item
#     diffusion_t=VDiffusion, # The diffusion method used
#     sampler_t=VSampler, # The diffusion sampler used
# )

In [5]:
dataset.data.shape[-1]

103

In [6]:
import lightning as lt

class Lightningwrapper(lt.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        loss = self.model(batch)
        self.log('train_loss', loss)
        return loss
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=3e-4)

#lt_model = Lightningwrapper(model)
dataloader = DataLoader(dataset, batch_size=64,num_workers=30,shuffle=True,)
# Train model with audio waveforms
trainer = lt.Trainer(max_epochs=40)
trainer.fit(model, dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params | Mode 
-------------------------------------------
0 | network | CAUnet | 375 M  | train
-------------------------------------------
375 M     Trainable params
0         Non-trainable params
375 M     Total params
1,501.201 Total estimated model params size (MB)
197       Modules in train mode
0         Modules in eval mode


Epoch 19: 100%|██████████| 54/54 [00:09<00:00,  5.86it/s, v_num=218]